In [11]:
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(' ')
enter_log = logging.getLogger('ENTER')
exit_log = logging.getLogger('EXIT')
start_log = logging.getLogger('START')
count_log = logging.getLogger('COUNT')
report_log = logging.getLogger('REPORT')

class Open:
    successful_counter = 0
    failed_counter = 0
    def __init__(self, name, mode='r', encoding='utf-8'):
        start_log.info(f'\nThe start operation with file <{name}> (mode="{mode}"):')
        self.name = name
        self.mode = mode
        self.encoding = encoding
        self.file = None

    @classmethod
    def successful_counter_add(cls):
        cls.successful_counter += 1
        count_log.info(f'SUCCESSFUL counter added to {cls.successful_counter}.')

    @classmethod
    def failed_counter_add(cls):
        cls.failed_counter += 1
        count_log.info(f'FAILED counter added to {cls.failed_counter}.')

    def __enter__(self):
        enter_log.info('Calling __enter__;')

        try:
            if self.mode != 'x':
                self.file = open(self.name, 'r', encoding=self.encoding)
                self.copy_file = self.file.read()
                enter_log.info(f'The file <{self.name}> was copied.')
                self.file.close()
        except BaseException as error:
            enter_log.error(f'Tried to read: {error.__class__} {error}')

        try:
            self.file = open(self.name, self.mode, encoding=self.encoding)
            enter_log.info(f'The file <{self.name}> was opened in "{self.mode}"-mode and returned.')
            return self.file
        except BaseException as error:
            enter_log.error(f'Tried to open in "{self.mode}": {error.__class__} {error}')
            self.failed_counter_add()
            raise error
        finally:
            self.report('ENTER')

    def __exit__(self, exc_type, exc_value, exc_traceback):
        exit_log.info('Calling __exit__;')

        if exc_type:

            exit_log.error(f'ERROR in EXIT: {exc_type} {exc_value} {exc_traceback}')

            self.failed_counter_add()

            if self.file:
                try:
                    with open(f'{self.name}', 'w', encoding=self.encoding) as new_file:
                        print(self.copy_file, file=new_file, end='')
                        exit_log.info(f'The file <{self.name}> was repaired successfully.')
                except BaseException as error:
                    exit_log.error(f'{error.__class__} {error}')
            self.report('EXIT (fail)')

            return True

        else:

            exit_log.info(f'The operation with file <{self.name}> (mode="{self.mode}") is successful.')
            self.successful_counter_add()

            self.report('EXIT (success)')

            return None

    def report(self, place):
        report_log.info(f'Report in {place}:')
        if self.file:
            report_log.info(f'Check 1: the file <{self.name}> is closed: {self.file.closed}.')
            if not self.file.closed:
                if place != 'ENTER':
                    self.file.close()
                    report_log.info(f'Check 2: the file <{self.name}> is closed: {self.file.closed}.')
                else:
                    report_log.info(f'Check 2: the file <{self.name}> is opened and sent to "EXIT".')
        else:
            report_log.info(f'The file <{self.name}> was not created.')

In [12]:
logging.info('\n\n----------1. Success(1) is expected:')
with Open('demo.txt', 'w') as opened_file:
    opened_file.write('PYTHON IS GREAT!')

INFO:root:

----------1. Success(1) is expected:
INFO:START:
The start operation with file <demo.txt> (mode="w"):
INFO:ENTER:Calling __enter__;
INFO:ENTER:The file <demo.txt> was copied.
INFO:ENTER:The file <demo.txt> was opened in "w"-mode and returned.
INFO:REPORT:Report in ENTER:
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is opened and sent to "EXIT".
INFO:EXIT:Calling __exit__;
INFO:EXIT:The operation with file <demo.txt> (mode="w") is successful.
INFO:COUNT:SUCCESSFUL counter added to 1.
INFO:REPORT:Report in EXIT (success):
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is closed: True.


In [13]:
logging.info('\n\n----------2. Success(2) is expected:')
with Open('demo.txt', 'r') as opened_file:
    logger.info(f'The file content: <<{opened_file.read()}>>.')

INFO:root:

----------2. Success(2) is expected:
INFO:START:
The start operation with file <demo.txt> (mode="r"):
INFO:ENTER:Calling __enter__;
INFO:ENTER:The file <demo.txt> was copied.
INFO:ENTER:The file <demo.txt> was opened in "r"-mode and returned.
INFO:REPORT:Report in ENTER:
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is opened and sent to "EXIT".
INFO: :The file content: <<PYTHON IS GREAT!>>.
INFO:EXIT:Calling __exit__;
INFO:EXIT:The operation with file <demo.txt> (mode="r") is successful.
INFO:COUNT:SUCCESSFUL counter added to 2.
INFO:REPORT:Report in EXIT (success):
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is closed: True.


In [14]:
try:
    logging.info('\n\n----------3. Fail(1) is expected:')
    with Open('demo1.txt', 'r') as opened_file:
        logger.info(f'The file content: <<{opened_file.read()}>>.')
except FileNotFoundError as error:
    logger.error(f'{error.__class__} {error}\n')

INFO:root:

----------3. Fail(1) is expected:
INFO:START:
The start operation with file <demo1.txt> (mode="r"):
INFO:ENTER:Calling __enter__;
ERROR:ENTER:Tried to read: <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'demo1.txt'
ERROR:ENTER:Tried to open in "r": <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'demo1.txt'
INFO:COUNT:FAILED counter added to 1.
INFO:REPORT:Report in ENTER:
INFO:REPORT:The file <demo1.txt> was not created.
ERROR: :<class 'FileNotFoundError'> [Errno 2] No such file or directory: 'demo1.txt'



In [15]:
try:
    logging.info('\n\n----------4. Fail(2) is expected:')
    with Open(123, 'r') as opened_file:
        print(opened_file.read())
except OSError as error:
    logger.error(f'{error.__class__} {error}\n')

INFO:root:

----------4. Fail(2) is expected:
INFO:START:
The start operation with file <123> (mode="r"):
INFO:ENTER:Calling __enter__;
ERROR:ENTER:Tried to read: <class 'OSError'> [WinError 6] The handle is invalid
ERROR:ENTER:Tried to open in "r": <class 'OSError'> [WinError 6] The handle is invalid
INFO:COUNT:FAILED counter added to 2.
INFO:REPORT:Report in ENTER:
INFO:REPORT:The file <123> was not created.
ERROR: :<class 'OSError'> [WinError 6] The handle is invalid



In [16]:
try:
    logging.info('\n\n----------5. Fail(3) is expected:')
    with Open('demo.txt', 'w') as opened_file:
        opened_file.write(wrong_context)
except NameError as error:
    logger.error(f'{error.__class__} {error}\n')

INFO:root:

----------5. Fail(3) is expected:
INFO:START:
The start operation with file <demo.txt> (mode="w"):
INFO:ENTER:Calling __enter__;
INFO:ENTER:The file <demo.txt> was copied.
INFO:ENTER:The file <demo.txt> was opened in "w"-mode and returned.
INFO:REPORT:Report in ENTER:
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is opened and sent to "EXIT".
INFO:EXIT:Calling __exit__;
ERROR:EXIT:ERROR in EXIT: <class 'NameError'> name 'wrong_context' is not defined <traceback object at 0x000001EEB3C30B00>
INFO:COUNT:FAILED counter added to 3.
INFO:EXIT:The file <demo.txt> was repaired successfully.
INFO:REPORT:Report in EXIT (fail):
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is closed: True.


In [17]:
logging.info('\n\n----------6. Success(3) is expected:')
with Open('demo.txt', 'a') as opened_file:
    opened_file.write('BEETROOT IS NICE!')

INFO:root:

----------6. Success(3) is expected:
INFO:START:
The start operation with file <demo.txt> (mode="a"):
INFO:ENTER:Calling __enter__;
INFO:ENTER:The file <demo.txt> was copied.
INFO:ENTER:The file <demo.txt> was opened in "a"-mode and returned.
INFO:REPORT:Report in ENTER:
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is opened and sent to "EXIT".
INFO:EXIT:Calling __exit__;
INFO:EXIT:The operation with file <demo.txt> (mode="a") is successful.
INFO:COUNT:SUCCESSFUL counter added to 3.
INFO:REPORT:Report in EXIT (success):
INFO:REPORT:Check 1: the file <demo.txt> is closed: False.
INFO:REPORT:Check 2: the file <demo.txt> is closed: True.


In [18]:
try:
    logging.info('\n\n----------7. Fail(4) is expected:')
    with Open('demo2.txt', 'x') as opened_file:
        opened_file.write('HELLO 2023!')
except FileExistsError as error:
    logger.error(f'{error.__class__} {error}\n')

INFO:root:

----------7. Fail(4) is expected:
INFO:START:
The start operation with file <demo2.txt> (mode="x"):
INFO:ENTER:Calling __enter__;
ERROR:ENTER:Tried to open in "x": <class 'FileExistsError'> [Errno 17] File exists: 'demo2.txt'
INFO:COUNT:FAILED counter added to 4.
INFO:REPORT:Report in ENTER:
INFO:REPORT:The file <demo2.txt> was not created.
ERROR: :<class 'FileExistsError'> [Errno 17] File exists: 'demo2.txt'



In [19]:
try:
    logging.info('\n\n----------8. Fail(5) is expected:')
    with Open('demo.txt', 'x') as opened_file:
        opened_file.write('SAMPLE TEXT...')
except FileExistsError as error:
    logger.error(f'{error.__class__} {error}\n')

INFO:root:

----------8. Fail(5) is expected:
INFO:START:
The start operation with file <demo.txt> (mode="x"):
INFO:ENTER:Calling __enter__;
ERROR:ENTER:Tried to open in "x": <class 'FileExistsError'> [Errno 17] File exists: 'demo.txt'
INFO:COUNT:FAILED counter added to 5.
INFO:REPORT:Report in ENTER:
INFO:REPORT:The file <demo.txt> was not created.
ERROR: :<class 'FileExistsError'> [Errno 17] File exists: 'demo.txt'



In [20]:
logger.critical(f'Successful_counter = {Open.successful_counter};')
logger.critical(f'Failed_counter = {Open.failed_counter};')

CRITICAL: :Successful_counter = 3;
CRITICAL: :Failed_counter = 5;
